# Environment

For this exercise, I'm running a virtualenv. Installing the pre-requisites should be accomplished using:

~~~ bash
pip install -r requirements.txt
~~~

From the root directory of this project.


## The additional ROOT Dependency

This is only to be used when reading from ROOT files. If the files are pre-converted to HDF5, then there's no need.

In [ ]:
#import ROOT

## Import block

I was looking into recreating this in tensorflow. It looks like the layer types are a little more extensible in Keras. Since Keras uses Tensorflow as a backend by default, I'll use Keras for some exploration. Later, I can come back to tensorflow.

In [ ]:
import logging
import h5py
from keras.layers import Convolution2D, MaxPooling2D  # Layer Def
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.utils.io_utils import HDF5Matrix
import h5py
import numpy as np
from collections import defaultdict
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# Exploring the input data

This should elucidate how data is being read out. I'm also going to use this to define which data to use as input and which data should be predicted.

For now, I'm just going to open up the file and print out the contents.

In [ ]:
data_file = h5py.File('chargedrec_mc7_phaseII_BGx1_KstZG.h5','r')

In [ ]:
[i for i in data_file]

[u'BtoKstG', u'BtoRhoG']

In [ ]:
BtoKstG = data_file[u'BtoKstG']
BtoKstG = data_file[u'BtoRhoG']

print BtoKstG

<HDF5 dataset "BtoRhoG": shape (1107437, 478), type "<f4">


It looks like the input data has the correct shape. There were 92790 events in the original ROOT file. Due to the non-flat 4-vector momenta, I extended the fields to include 4 fields for each 4-vector. Thus the second index is a bit bigger than the number of fields in the trees in the original root file.

As a matter of convenience, I store the names of the columns in the tensors as values in the attributes field corresponding to `index<>_name`, where `<>` denotes the column number.

In [ ]:
for key in BtoKstG.attrs.keys():
    print key,": ", BtoKstG.attrs[key]

index0_name :  exp_no
index1_name :  run_no
index2_name :  evt_no
index3_name :  nECLClusters
index4_name :  nNeutralECLClusters
index5_name :  nChargedECLClusters
index6_name :  nGoodNeutralECLClusters
index7_name :  neutralECLEnergy
index8_name :  chargedECLEnergy
index9_name :  goodNeutralECLEnergy
index10_name :  nTracks
index11_name :  nMCParticles
index12_name :  nParticles
index13_name :  B0_nROETracks
index14_name :  B0_nROEECLClusters
index15_name :  B0_nGoodROEClusters
index16_name :  B0_nROEKLMClusters
index17_name :  B0_deltae
index18_name :  B0_mbc
index19_name :  B0_ThrustB
index20_name :  B0_ThrustO
index21_name :  B0_CosTBTO
index22_name :  B0_CosTBz
index23_name :  B0_R2
index24_name :  B0_cc1
index25_name :  B0_cc2
index26_name :  B0_cc3
index27_name :  B0_cc4
index28_name :  B0_cc5
index29_name :  B0_cc6
index30_name :  B0_cc7
index31_name :  B0_cc8
index32_name :  B0_cc9
index33_name :  B0_mm2
index34_name :  B0_et
index35_name :  B0_hso00
index36_name :  B0_hso01
i

## Input and prediction data description

The idea here is that I'm not sure how I'm going to define which fields are "measured" data, be they mc or real data, and which fields are "truths" or "labels". So, what I'd like to do is, for now, define a rank-2 list in python with the corresponding keys to each of the fields. In a couple steps, when I define the data generator, I can use these to stitch together the input matrix for the network.

Obviously, this can be done better by preparing the data in file more sensibly, but this is proof of concept, so I'm less worried about efficiency or elegence.

In [ ]:
input_data_desc=[
    ['B0_R2','B0_CosTBTO'],
    ['B0_KST0_pi__piid','B0_KST0_K__piid'],
    ['B0_KST0_K_P','B0_KST0_pi_P'],
    ['nTracks','B0_gamma_clusterE9E25'],
    ['nGoodNeutralECLClusters','B0_gamma_CosTBTO'],
    ['B0_KST0_K_d0','B0_KST0_K_z0'],
    ['B0_KST0_pi_d0','B0_KST0_pi_z0'],
]

output_data_desc=['B0__isSignal']

In [ ]:
class B2DataGenerator(object):
    """
        TODO: This can and should be threaded out.
    """
    batch_size = 10
    logger = logging.getLogger("b2.data")
    def __init__(self, datapath, dataset, input_data_desc, output_data_desc):
        self.logger.info("Assembling DataSet")
        self._file = h5py.File(datapath,'r')
        self._dataset = self._file[dataset]
        
        self._data_desc=input_data_desc
        self.output_data_desc = output_data_desc
        
        #book keeping
        self.current_index=0
        
    def output(self):
        """
            Returns an input layer with the appropriate shape to
            match the data that we're using.
            
            Note, that this assumes that shape of the input tensor is hyper-rectangular.
            That is to say, NO STAGGERED MATRICES. Yet. I still have to figure that part out.
        """
        return Input(shape=(len(self._data_desc),len(self._data_desc[0]),1))

    def __len__(self):
        """
        This assumes all of the data is stored in rows
        """
        return self._dataset.shape[0]

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        """
            This is still a largely inefficient way of doing this.
            TODO: fix that
        """
        batch = self._dataset[self.current_index:self.current_index+self.batch_size]
        indices=[]
        prediction_index = -1
        for row in self._data_desc:
            for col in row:
                for key in self._dataset.attrs:
                    if self._dataset.attrs[key] == col:
                        indices.append(int(key.split('x')[1].split('_')[0]))
                    elif self._dataset.attrs[key] == self.output_data_desc[0]:
                        prediction_index = int(key.split('x')[1].split('_')[0])
        data = batch[:, tuple(indices)]
        data = data.reshape( (self.batch_size,len(self._data_desc),len(self._data_desc[0]),1) )

        prediction = batch[:, prediction_index]
        self.current_index+=self.batch_size
        return (data, prediction)

In [ ]:
BtoKstG_datagen = B2DataGenerator('chargedrec_mc7_phaseII_BGx1_mixed.h5','BtoKstG',input_data_desc, output_data_desc)

# Model Definition

For this excercise, I'm using vgg since it's a fairly lightweight network. I also had this prepped from an earlier study, so extending it to work for this data should be easy

In [ ]:
class VGG16(Model):
    logger = logging.getLogger('b2.vgg16')
    def __init__(self, input_data,loss='binary_crossentropy'):
        self._input = input_data
        self.logger.info("Assembling Model")

        self.logger.debug("Input Shape: {}".format(self._input))

        # Block 1
        layer = Convolution2D(64, 2, 2, activation='relu', border_mode='same', 
                              name='block1_conv1')(self._input)
        layer = Convolution2D(64, 2, 2, activation='relu', border_mode='same', 
                              name='block1_conv2')(layer)
        layer = MaxPooling2D((2, 1), strides=(1, 1), name='block1_pool')(layer)

        # Block 2
        layer = Convolution2D(128, 3, 2, activation='relu', border_mode='same', 
                              name='block2_conv1')(layer)
        layer = Convolution2D(128, 3, 2, activation='relu', border_mode='same', 
                              name='block2_conv2')(layer)
        layer = MaxPooling2D((2, 1), strides=(1, 1), name='block2_pool')(layer)


        # Classification block
        layer = Flatten(name='flatten')(layer)
        layer = Dense(4096, activation='relu', name='fc1')(layer)
        layer = Dense(4096, activation='relu', name='fc2')(layer)
        layer = Dense(1, activation='softmax', name='predictions')(layer)

        super(VGG16, self).__init__(self._input, layer)
        self.logger.info("Compiling Model")
        self.compile(loss=loss, optimizer='sgd')

## Modifying the loss function

Previously, we were using binary crossentropy as the loss function. Now, we'd like to motivate loss by physics.

Below is an example of a loss function implementing 

In [ ]:
_EPSILON = K.epsilon()

def loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    out = -(y_true * K.log(y_pred) + (1.0 - y_true) * K.log(1.0 - y_pred))
    return K.mean(out, axis=-1)

def loss_np(y_true, y_pred):
    y_pred = np.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    out = -(y_true * np.log(y_pred) + (1.0 - y_true) * np.log(1.0 - y_pred))
    return np.mean(out, axis=-1)

# Training!

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logging.info("Starting...")

model = VGG16(BtoKstG_datagen.output())
training_output = model.fit_generator(BtoKstG_datagen, samples_per_epoch = 1000, nb_epoch=1000)

INFO:root:Starting...
INFO:b2.vgg16:Assembling Model
DEBUG:b2.vgg16:Input Shape: Tensor("input_1:0", shape=(?, 7, 2, 1), dtype=float32)
INFO:b2.vgg16:Compiling Model


Epoch 1/1000
1000/1000 [==============================] - 99s - loss: 15.9264      

In [ ]:
print dir(training_output)

## Training Metrics

Given the training output, you can plot some of the metrics inline. For instace, here is the loss history plotted 

In [ ]:
plt.figure()
plt.plot(training_output.epoch, training_output.history['loss'])

In [ ]:
print training_output.params

In [ ]:
print BtoKstG_datagen.next()

In [ ]:
x,y = BtoKstG_datagen.next()

In [ ]:
print y

In [ ]:
print x

In [ ]:
print x.shape